In [ ]:
%matplotlib inline

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.contrib.distributions import Bernoulli
from IPython.display import Image

In [ ]:
!ls "../.."

In [ ]:
import gensim
import pandas as pd
from utils import *

from libKMCUDA import kmeans_cuda
from sklearn.metrics import accuracy_score

#model = gensim.models.Word2Vec.load("assets/doc2vecModels/KTH2004_i10000_w10_d500_plainTrain/KTH2004_i10000_w10_d500_plainTrain.model")
#model = gensim.models.Word2Vec.load("assets/doc2vecModels/KTH2004_i2000_w10_d500_plainTrain_small/KTH2005_i2000_w10_d500_plainTrain")
model = gensim.models.Word2Vec.load("../../assets/doc2vecModels/KTH2004_i2000_w10_d500_plainTrain_small_noAuth/KTH2005_i2000_w10_d500_plainTrain_noAuth")
doc_vec = model.docvecs.vectors_docs
doc_tag = list(model.docvecs.doctags.keys())
word_vectors = model.wv.vectors

pickle_o = pickle_obj(); 
id_to_auth = pickle_o.load("../../assets/dictionaries/id_to_all_auths_2004")
auth_to_id = pickle_o.load("../../assets/dictionaries/auths_to_all_id_2004")
id_to_auth = pickle_o.load("../../assets/dictionaries/id_to_all_auths_2004")
auth_to_id = pickle_o.load("../../assets/dictionaries/auths_to_all_id_2004")

negative_test = pickle_o.load("../../assets/goldenstandards/test_neg_aricles")
negative_pos = pickle_o.load("../../assets/goldenstandards/test_pos_aricles")

df_auth = pd.read_csv("../../assets/dataframes/KT_auth_2004")
df_abs = pd.read_csv("../../assets/dataframes/all_authors_df_2004")

#negative_test = pickle_o.load("assets/goldenstandards/test_neg_aricles")
#negative_pos = pickle_o.load("assets/goldenstandards/test_pos_aricles")

negative_val = np.load("../../assets/goldenstandards/NLS_val.npy")
negative_test = np.load("../../assets/goldenstandards/NLS_test.npy")
postive_val = np.load("../../assets/goldenstandards/LS_val.npy")
positve_test = np.load("../../assets/goldenstandards/LS_test.npy")


tes_tag_str = list(np.array(list(negative_val) + list(negative_test)+list(postive_val)+list(positve_test)).astype(str))
train_tag = list(set(doc_tag) - (set(tes_tag_str)))
train_vec = model[train_tag]

test_nls = list(np.asarray(negative_test).astype(str))
val_nls = list(np.asarray(negative_val).astype(str))
test_ls = list(np.asarray(positve_test).astype(str))
val_ls = list(np.asarray(postive_val).astype(str))

X_nls_test = model[test_nls]
X_nls_val = model[val_nls]

X_ls_test = model[test_ls]
X_ls_val = model[val_ls]


y_nls_test = np.ones(X_nls_test.shape[0])
y_nls_val = np.ones(X_nls_val.shape[0])

y_ls_test = np.zeros(X_ls_test.shape[0])
y_ls_val = np.zeros(X_ls_val.shape[0])


X_test = np.concatenate((X_nls_test, X_ls_test))
X_val = np.concatenate((X_nls_val, X_ls_val))

y_test = np.concatenate((y_nls_test, y_ls_test))
y_val = np.concatenate((y_nls_val, y_ls_val))

all_vec = np.concatenate([train_vec, word_vectors])
tags =np.concatenate([train_tag, model.wv.index2word])
all_vec = np.asarray(all_vec)
tags =np.asarray(tags)

col_names = pd.read_csv('../../assets/clusterModels/assignKMM_clusters1000Articles', nrows=0).columns
types_dict = {}
types_dict.update({col: str for col in col_names if col not in types_dict})
df = pd.read_csv("../../assets/clusterModels/assignKMM_clusters1000Articles", dtype=types_dict)
df.drop(["Unnamed: 0"],axis=1, inplace=True)


col_names = pd.read_csv('../../assets/clusterModels/assignKMM_clusters1000Articles_clusters_with_only_authors', nrows=0).columns
types_dict = {}
types_dict.update({col: str for col in col_names if col not in types_dict})
df = pd.read_csv("../../assets/clusterModels/assignKMM_clusters1000Articles_clusters_with_only_authors", dtype=types_dict)
df.drop(["Unnamed: 0"],axis=1, inplace=True)

col_names = pd.read_csv("../../assets/clusterModels/assignKMM_clusters300Articles_df_cluster_authors", nrows=0).columns
types_dict = {}
types_dict.update({col: str for col in col_names if col not in types_dict})

df_cluster_authors = pd.read_csv("../../assets/clusterModels/assignKMM_clusters300Articles_df_cluster_authors", dtype=types_dict)
df_cluster_authors.drop(["Unnamed: 0"],axis=1, inplace=True)
df_cluster_authors.columns = df_cluster_authors.columns.astype(int)

postive_clusters = pickle_o.load("../../assets/clusterModels/KMM1000_testVal_postive_clusters")
negaitves_clusters = pickle_o.load("../../assets/clusterModels/KMM1000_testVal_negaitves_clusters")

p_list_cluster = list(set([item[0] for sublist in postive_clusters for item in sublist]))
n_list_cluster = list(set([item[0] for sublist in negaitves_clusters for item in sublist]))
intersect = list(set(n_list_cluster) & set(p_list_cluster))
n_list_cluster = list(set(n_list_cluster) ^ set(intersect))

not_NLS = [667, 759, 118, 237, 779, 815, 166, 716, 832, 771, 400, 200, 515, 691, 357, 19, 29, 333, 227, 277]
extra_ls = [759, 118, 237, 779, 716, 832, 771, 400, 200, 691, 357, 333, 227, 277]
LS = [115, 329, 423, 680, 562, 45, 178, 624, 143, 602, 784, 748, 788, 763, 645, 328, 564, 919, 444, 892, 962
     , 24, 632, 160, 187, 936, 972, 76, 163, 533, 841, 230, 416]
NLS = list((set(n_list_cluster) - set(not_NLS)))
#LS = LS + extra_ls
potential_ls = list((set(df.columns.values.astype(int)) - set(LS)) - set(NLS))

#NLF_train_v, nlf_train_lab = get_labels_and_vec(df, NLS, 1)
#LF_train_v, lf_train_lab = get_labels_and_vec(df, LS, 1)

NLF_train_v, nlf_train_lab = get_train_data(df, NLS, model, 1)
LF_train_v, lf_train_lab = get_train_data(df, LS, model, 1)

X_train, y_train = concatenate_and_get_labels(LF_train_v, NLF_train_v)
X_un_pot, L_pot = getVectors(df, model, potential_ls)

In [ ]:
!python3 -c "import tensorflow as tf; print(tf.__version__)"

In [ ]:
class StochasticLayer:
    """
    StochasticLayer Dense Layer.

    Parameters
    ----------
    BN : bool
        Batchnormalization
    n_in : int
        Input nodes
    n_out: int
        Output nodes
    model_prob: float
        Dropout probability
    model_lam: float
        regualarization term
    is_training: bool
        traing-phase/test-phase        
    
    Returns
    -------
    array
        Outputlayer
    """
    

    def __init__(self, n_in, n_out, model_prob, model_lam, is_training, BN=False, decay=0.99):
        
        
    
        self.is_training = is_training    
        self.model_prob = model_prob
        self.model_lam = model_lam
        self.model_bern = Bernoulli(probs=self.model_prob, dtype=tf.float32)
        self.model_M = tf.Variable(tf.truncated_normal([n_in, n_out], stddev=0.01))
        self.model_m = tf.Variable(tf.zeros([n_out]))
        self.model_W = tf.matmul(
            tf.diag(self.model_bern.sample((n_in, ))), self.model_M
        )
        #Parameters for BatchNormalization
        self.BN = BN
        self.scale = tf.Variable(tf.ones([n_out]))
        self.beta = tf.Variable(tf.zeros([n_out]))
        self.epsilon = 1e-2
        self.mean = tf.Variable(tf.zeros([n_out]), trainable=False)
        self.var = tf.Variable(tf.ones([n_out]), trainable=False)
        self.decay = decay

    def __call__(self, X, activation=tf.identity):
        output = activation(tf.matmul(X, self.model_W) + self.model_m)
        
        if self.BN:
            output = self.batch_norm_wrapper(output, decay = 0.5)
                
        if self.model_M.shape[1] == 1:
            #output = tf.squeeze(output)
            output = output
        return output

    @property
    def regularization(self):
        """regularization"""

        return self.model_lam * (
            self.model_prob * tf.reduce_sum(tf.square(self.model_M)) +
            tf.reduce_sum(tf.square(self.model_m))
        )

    
    
    def batch_norm_wrapper(self, inputs, decay = 0.99):
        """
        Batchnormalization

        Parameters
        ----------
        inputs : array
            Batch input
        is_training : bool
            training phase/test phase
        decay: float
            Decrease training of popoulation mean and variance.    
        Returns
        -------
        array
            Normalized batched
        """
        
        
        
        def BN_train():
            """
            Batchnormalization training
 
            Returns
            -------
            array
                Normalized train batch
            """
            batch_mean, batch_var = tf.nn.moments(inputs,[0])
            train_mean = tf.assign(self.mean, self.mean * self.decay + batch_mean * (1 - self.decay))
            train_var = tf.assign(self.var, self.var * self.decay + batch_var * (1 - self.decay))
            #train_mean = tf.assign(self.mean, self.mean + batch_mean)
            #train_var = tf.assign(self.var, self.var + batch_var)
            
            with tf.control_dependencies([train_mean, train_var]):
                return tf.nn.batch_normalization(inputs,
                    batch_mean, batch_var, self.beta, self.scale, self.epsilon)
            
        def BN_test():
            """
            Batchnormalization test
 
            Returns
            -------
            array
                Normalized test batched
            """
            return tf.nn.batch_normalization(inputs,
                self.mean, self.var, self.beta, self.scale, self.epsilon)
        
        
        return tf.cond(self.is_training, BN_train, BN_test)

In [ ]:
import shutil, os, glob
 
def moveAllFilesinDir(srcDir, dstDir):
    # Check if both the are directories
    if os.path.isdir(srcDir) and os.path.isdir(dstDir) :
        # Iterate over all the files in source directory
        for filePath in glob.glob(srcDir + '\*'):
            print(filePath)
            # Move each file to destination Directory
            shutil.move(filePath, dstDir);
    else:
        print("srcDir & dstDir should be Directories")

In [ ]:
class BayesDNNCalssifier:
    def __init__(self, batchnorm=True, dropout_fit=0.8, dropout_sample=0.8, batch_size=256, mc_samples=800):
        self.BN = batchnorm
        self.dropout_fit = dropout_fit
        self.dropout_sample = dropout_sample
        self.bs = batch_size
        self.load = False
        self.mc_samples = mc_samples
    def _build_graph(self, n_feats, n_samples, dp_prob=1.0, BN=False, decay=0.99):
        """
        Building training graph

        Parameters
        ----------
        dp_prob : float
            Dropout prob
        BN : bool
            Batchnorm
        Returns
        -------
        array
            graph outputs
        """
    
        #n_feats = X.shape[1]
        n_hidden = 500
        model_prob = dp_prob
        model_lam = 1e-8
        BN=BN
    
        X_input = tf.placeholder(tf.float32, [None, n_feats])
        y_input = tf.placeholder(tf.float32, [None,1])
    
        is_training = tf.placeholder(tf.bool)
    
        Layer_1 = StochasticLayer(n_feats, n_hidden * 2, model_prob, model_lam, is_training ,BN=BN, decay=decay)
        Layer_2 = StochasticLayer(n_hidden * 2, n_hidden, model_prob, model_lam, is_training,BN=BN, decay=decay)
        Layer_3 = StochasticLayer(n_hidden, n_hidden // 2, model_prob, model_lam, is_training,BN=BN, decay=decay)
        Layer_4 = StochasticLayer(n_hidden // 2, 1, model_prob, model_lam, is_training, BN=BN, decay=decay)
    
        z_1 = Layer_1(X_input, tf.nn.sigmoid)
        z_2 = Layer_2(z_1, tf.nn.sigmoid)
        z_3 = Layer_3(z_2, tf.nn.sigmoid)
        y_pred = Layer_4(z_3, tf.nn.sigmoid)
    
        cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=y_input, logits=y_pred)
        loss = tf.reduce_mean(cross_entropy)
    
        predicted = tf.nn.sigmoid(y_pred)

        correct_pred = tf.equal(tf.round(predicted), y_input)
        accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    
        model_loss = (
            # Negative log-likelihood.
            loss +
            # Regularization.
            Layer_1.regularization +
            Layer_2.regularization +
            Layer_3.regularization
                    ) / n_samples
    
        train_step = tf.train.AdamOptimizer(1e-3).minimize(model_loss)

        
        return (X_input, y_input), train_step, accuracy, model_loss, predicted, tf.train.Saver(), is_training

    
    def _train(self, X, y, train_step, X_input, y_input, is_training, accuracy, model_mse, saver, 
               epoch=20, verbose =True, bs=10):
        """
        Train model

        Parameters
        ----------
        iterations : int
            Number of iterations
        verbose : bool
            Verbose
        bs: int
            batch-size
        """
        with tf.Session() as sess:
            if self.load:
                saver.restore(sess, self.path)
                
            sess.run(tf.global_variables_initializer())
            for e in range(epoch):
                test_idx = np.arange(0 , len(X))
                np.random.shuffle(test_idx)
                for i in range(0,len(X), bs): 
                    X_batch = X[test_idx[i:(i+bs)]]
                    y_batch = y[test_idx[i:(i+bs)]]
            
                    train_step.run(feed_dict={X_input: X_batch, y_input: y_batch, is_training:True})
                if verbose:
                    print(e)
                    mse = sess.run(model_mse, {X_input: X_batch, y_input: y_batch, is_training:True})
                    _accuracy = sess.run(accuracy, {X_input: X_batch, y_input: y_batch, is_training:True})
                    print("Epoch {}, -log(mse): {:.2f} , acc: {:.2f}".format(e, -np.log(mse), _accuracy))
            
            self.load = False
            saved_model = saver.save(sess, 'temp/temp-bn-save')
            sess.close()
   
    def fit(self,X, y, epoch):
        self.X_train = X
        tf.reset_default_graph()
        (X_input, y_input), train_step, accuracy, model_mse, _, saver, is_training = self._build_graph(X.shape[1],
                                                                                                       self.bs,
                                                                                                       BN=self.BN, 
                                                                                                       dp_prob=self.dropout_fit)
        self.saver = saver
        self._train(X, y.reshape(-1,1), train_step, X_input, y_input, is_training,
                    accuracy, model_mse, saver, epoch=epoch, verbose = True, bs=self.bs)
    
    def save_weights(self):
        path = "/home/ekvall/kth-cluster/kth-cluster/getLifeScience/BayesDNN/Feedforward_example/"
        if os.path.isdir(path + "bestmodel/temp"):
            shutil.rmtree(path + "bestmodel/temp")
            
        shutil.move(path + "temp",
                   path + "bestmodel/")
        
    def load_weights(self):
        self.load = True
        self.path = "/home/ekvall/kth-cluster/kth-cluster/getLifeScience/BayesDNN/Feedforward_example/bestmodel/temp/temp-bn-save"

        
    def _test(self, X_pred, X_input, y_input, train_step, model_pred, saver, is_training,
              bs=10,BN=False):
        """
        Test model

        Parameters
        ----------
        samples : int
            Monte Carlo Samples
        bs : int
            Batchnorm samples
        """
    
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            if self.load:
                saver.restore(sess, self.path)
                print("yo")
            else:
                saver.restore(sess, 'temp/temp-bn-save')
    

            Y_sample = np.zeros((self.mc_samples, X_pred.shape[0]))
            for i in range(self.mc_samples):       
                if BN:
                    test_idx = np.arange(0 , len(X_pred))
                    np.random.shuffle(test_idx)
                    model_pred.eval({X_input: self.X_train[test_idx[:bs]],  is_training: True})
                Y_sample[i] = (sess.run(model_pred, {X_input: X_pred, is_training: False})).ravel()
            return Y_sample
    
    def sample_predictions(self, X_pred):
        tf.reset_default_graph()
        (X_input, y_input), train_step, accuracy, model_mse, model_pred, saver, is_training = self._build_graph(X_train.shape[1],
                                                                                                                self.bs,
                                                                                                                BN=self.BN, 
                                                                                                          dp_prob=self.dropout_sample)
        Y_sample = self._test(X_pred, X_input, y_input, train_step, model_pred, saver, is_training,
                              bs=self.bs, BN=False)
        
        return Y_sample.round()
    
    def predict(self, X_pred):
        y_samples = self.sample_predictions(X_pred)
        pred = y_samples.sum(axis=0)
        pred[pred <= self.mc_samples //2] = 0
        pred[pred > self.mc_samples //2] = 1
        return pred
    
    def probability(self, X_train, X_pred):
        n_1 = self.sample_predictions(X_train, X_pred).sum(axis=0)
        N = n_1.copy()
        n_0 = self.mc_samples - n_1
        N[n_0 >= n_1] = n_0[n_0 >= n_1]
        P = N / self.mc_samples * 100
        return P
        
        
        
        
        

In [ ]:
dnn = BayesDNNCalssifier(batchnorm=True, dropout_fit=0.7, dropout_sample=0.6, batch_size=256)

In [ ]:
dnn.fit(X_train, y_train, 10)

In [ ]:
pred_test = dnn.predict(X_test)

In [ ]:
pred_val = dnn.predict(X_val)

In [ ]:
dnn.load_weights()

In [ ]:
dnn.save_weights()

In [ ]:
!mkdir bestmodel

In [ ]:
!ls "bestmodel/temp/"

In [ ]:
dnn.load

In [ ]:
accuracy_score(pred_val, y_val)

In [ ]:
accuracy_score(pred_test, y_test)

In [ ]:
correct_ix = pred == y_val
false_ix = np.invert(correct_ix)

In [ ]:
pred_pre[false_ix]

In [ ]:
P[false_ix]

In [ ]:
np.mean(P)

In [ ]:
accuracy_score(pred_pre,y_val)

In [ ]:
Y_sample.astype(int).sum(axis=0)

In [ ]:
#from IPython.display import Image
#Image(filename="./assets/point_estimate.jpg") 

# Bayesian - Dropout


Credit: 

Article: Dropout as a Bayesian Approximation: Representing Model Uncertainty in Deep Learning

Authors: Yarin Gal, Zoubin Ghahramani

In [ ]:
#sess.close()
tf.reset_default_graph()
(X_input, y_input), train_step, model_mse, _, saver, is_training = build_graph(BN=False, dp_prob=0.9)
train(verbose = False, bs=n_samples)

In [ ]:
#sess.close()
tf.reset_default_graph()
(X_input, y_input), train_step, model_mse, model_pred, saver, is_training = build_graph(BN=False, dp_prob=0.9)
Y_sample_DP = test(mc_samples, bs=n_samples)
plot(Y_sample_DP, X_pred, mc_samples,color_line="b-", path="./assets/non_DO_BN")

# Bayesian - BatchNormalization 


Credit:

Aricle: Bayesian Uncertainty Estimation for Batch Normalized Deep Networks

Authors: Mattias Teye, Hossein Azizpour, Kevin Smith

In [ ]:

tf.reset_default_graph()
(X_input, y_input), train_step, model_mse, _, saver, is_training = build_graph(BN=True, dp_prob=1.0, decay=0.6)
train(verbose = False , bs=15)

In [ ]:

tf.reset_default_graph()
(X_input, y_input), train_step, model_mse, model_pred, saver, is_training = build_graph(BN=True, dp_prob=1.0, decay=0.6)
Y_sample_BN = test(mc_samples, bs=15,BN=True)
plot(Y_sample_BN, X_pred,mc_samples, color_line="g-",path="./assets/BN")


In [ ]:
if True:
        plt.figure(figsize=(8,6))
        for i in range(mc_samples):
            plt.plot(X_pred[:, 0], Y_sample_BN[i], "b-", alpha=1. / 200)
        for i in range(mc_samples):
            plt.plot(X_pred[:, 0], Y_sample_DP[i], "r-", alpha=1. / 200)
        plt.plot(X[:, 0], y, "g.")
        
        
        plt.grid()
        plt.show()
        #if path:
        #    plt.savefig(path, format='eps', dpi=1000)

# Bayesian - BatchNormalization and Drop out

In [ ]:
#sess.close()
tf.reset_default_graph()
(X_input, y_input), train_step, model_mse, _, saver, is_training = build_graph(BN=True, dp_prob=0.85, decay=0.5)
train(verbose = False, bs=20)

In [ ]:
#sess.close()
tf.reset_default_graph()
(X_input, y_input), train_step, model_mse, model_pred, saver, is_training = build_graph(BN=True, dp_prob=0.85, decay=0.5)
Y_sample = test(mc_samples, bs=20,BN=True)
plot(Y_sample, X_pred,mc_samples, color_line="b-",path="./assets/BN_DO")